Relations take the form $R(A, B, ...)$ where
- $R$ is the name of the relation
- $A, B, ...$ is the set of attributes of the relation
  - Often write the set without commas: $A, B, ... \equiv AB ...$, and can 
    refer to a set of attributes as $\vec{A}$ (vector A)
  - The number of attributes n is the arity of the relation
    Can call $R(A_1, ... A_n)$ an n-ary relation
  - $Domain(A)$ is the set of values (type) that the attribute can have
  - Will use $Attrs(R)$ to find $A, B, ...$

  

- The extent of $R(A, B, ...)$ is the set of tuples: 
        $\{<v^A_{1}, v^B_{1}, \ldots>, <v^A_{2}, v^B_{2} \ldots>, <v^A_{3}, v^B_{3} \ldots>\}$
    - $\forall.v^A_{x} \in Domain(A)$
    - No duplicate tuples
    - Not ordered
    - All tuples have the same arity



### Set Semantics

- Order of columns not significant
- Order of rows not significant
- No duplicate rows

---

- Attribute $=$ Column
- Tuple     $=$ Row